In [1]:
# 可视化GTE
import cv2 as cv
from skimage import measure
from scipy.ndimage import rotate
import numpy as np
import os.path as osp
import os
import torch
import shutil



IMG_SIZE = 2048
OFFSET = 1
MAX_DEGREE = 6
VECTOR_NORM = 25.0

NUM_PROCESS = 10
NUM_THREAD = 10
 
# 将图编码为19维张量
rgb_pattern   = './cityscale/20cities/region_{}_sat.png'
GT_GTE_logits_pattern = './cityscale/GTE/region_{}_GTE.npz' # 真实的GTE
GTE_logits_pattern = './save/GTE_修正损失计算重训_37epoch/GTE_logits/region_{}_GTE_logits.npz' # 预测的GTE



def vis_GT_GTE(GTE, keypoint_thr=0.1, edge_thr=0.1, aug=False, rot_angle=90, rot_index=0, pred=True):
    vis_output = np.zeros((IMG_SIZE, IMG_SIZE, 3))    # 不加底图
    # vis_output = cv.imread(rgb_pattern.format(7))[:, :, :]
    sub_GTE = torch.tensor(GTE[:, :, :], dtype=torch.float32)
    
    if aug:
        sub_GTE = GTE[:, :, :]
        rot_mat = np.array([    # 只针对90度的旋转，如果是任意角度还需要改旋转矩阵
                        [0, 1],
                        [-1, 0]
            ], dtype=np.float32)

        sub_GTE = rotate(sub_GTE, rot_angle*rot_index, axes=(0, 1), reshape=False)
        vis_output = rotate(vis_output, rot_angle*rot_index, axes=(0, 1), reshape=False)
        
        # 只对所有不为0的进行变换，其余的全是0，没必要再计算了
        for r, c in np.column_stack(np.where(sub_GTE[:,:, 0]>0.98)):
            delta_coords = []
            for j in range(MAX_DEGREE):
                delta_coords.append([sub_GTE[r, c, 1+3*j+1], sub_GTE[r, c, 1+3*j+2]])   # dx, dy
            delta_coords_to_rot = np.column_stack(delta_coords)
            roted_coords = np.linalg.matrix_power(rot_mat, rot_index)@delta_coords_to_rot
            
            for j in range(MAX_DEGREE):    # stick back
                sub_GTE[r, c, 1+3*j+1], sub_GTE[r, c, 1+3*j+2] = roted_coords[:, j]
                
    keypoint_map = sub_GTE[:,:, 0].detach().cpu().numpy()

    # 从团中寻找中心点
    keypoint_map = (keypoint_map > keypoint_thr).astype(np.uint8)
    # cv.imwrite(osp.join(verify_dir, f'region_8_vis_{rot_angle*rot_index}.png'), vis_output)
    # cv.imwrite(osp.join(verify_dir, f'region_8_vis_{rot_angle*rot_index}.png'), keypoint_map*255)
    labels = measure.label(keypoint_map, connectivity=2)
    props = measure.regionprops(labels)
    min_area = 0
    for region in props:
        if region.area > min_area:
            center = region.centroid[::-1]   # rc-> xy
            center_x, center_y = int(center[0]), int(center[1])
            cv.circle(vis_output, (center_x, center_y), radius=2, color=(0, 0, 255), thickness=-1)
            
            r, c = center_y, center_x
            for j in range(MAX_DEGREE):
                edgeness = torch.sigmoid(sub_GTE[r, c, 1+3*j])
                if edgeness > edge_thr:
                    dx, dy = sub_GTE[r, c, 1+3*j+1], sub_GTE[r, c, 1+3*j+2]
                    # dx, dy = rot_mat@np.array([dx, dy])
                    dst_x, dst_y = int(center_x + VECTOR_NORM*dx), int(center_y + VECTOR_NORM*dy)
                    cv.line(vis_output, (center_x, center_y), (dst_x, dst_y), color=(0, 255, 0), thickness=1)
    suffix = 'pred' if pred else 'GT'
    cv.imwrite(osp.join(verify_dir, f'region_8_vis_{rot_angle*rot_index}_{suffix}.png'), vis_output)
    


pred_GTE = np.load(open(GTE_logits_pattern.format(179), 'rb'))['GTE_logits']
GT_GTE = np.load(open(GT_GTE_logits_pattern.format(179), 'rb'))['GTE']

pred_keypoint_map = pred_GTE[:,:,0]
GT_keypoint_map = GT_GTE[:,:,0]

pred_idx = np.where(pred_keypoint_map)
GT_idx = np.where(GT_keypoint_map)

print('pred_keypoint_map 统计数据： mean, min, max')
print(pred_keypoint_map[pred_idx].mean(), pred_keypoint_map[pred_idx].min(), pred_keypoint_map[pred_idx].max())

print('GT_keypoint_map 统计数据： mean, min, max')
print(GT_keypoint_map[GT_idx].mean(), GT_keypoint_map[GT_idx].min(), GT_keypoint_map[GT_idx].max())


verify_dir = './save/成功的GTE/verify'
# if osp.exists(verify_dir):
#     shutil.rmtree(verify_dir)
# os.makedirs(verify_dir)
# vis_GT_GTE(pred_GTE, aug=False, pred=True)
# vis_GT_GTE(GT_GTE, aug=False, pred=False)

pred_keypoint_map 统计数据： mean, min, max
0.00472353 2.030601e-10 0.78709155
GT_keypoint_map 统计数据： mean, min, max
1.0 1.0 1.0


In [2]:
# 用Sat2Graph的解码算法解码一下
from decoder import DecodeAndVis

output_dir = './save/成功的GTE/decode_result'
pred_decode_file = osp.join(output_dir, f'region_{179}')
GT_decode_file = osp.join(output_dir, f'GT_region_{179}')

pred_graph = DecodeAndVis(pred_GTE, pred_decode_file, thr=0.05, edge_thr=0.05, angledistance_weight=10, snap=True, imagesize=2048)
# Gt_graph = DecodeAndVis(GT_GTE, GT_decode_file, thr=0.05, edge_thr=0.1, angledistance_weight=50, snap=True, imagesize=2048)

内插了256个点


/home/godx/research/source_code/sam_road/douglasPeucker.py:18: RuntimeWarning: invalid value encountered in divide
  return area/l


In [ ]:

import json
import pickle
import cv2 as cv
from PIL import Image
import os, shutil
import numpy as np



pred_graph_pattern = './save/修正损失计算重训/decode_result_origin/region_{}_graph.p'

save_dir = './save/修正损失计算重训/skel_from_graph'
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
os.makedirs(save_dir)
save_pattern = os.path.join(save_dir, '{}.png')

test_idxs = json.load(open('./cityscale/data_split.json', 'rb'))['test']


for idx in test_idxs:
    pred_graph_path = pred_graph_pattern.format(idx)
    
    sat = np.zeros((2048, 2048), dtype=np.uint8)
    pred_graph = pickle.load(open(pred_graph_path, 'rb'))
    
    for n, v in pred_graph.items():
        src_y, src_x = n
        for nei in v:
            dst_y, dst_x = nei
            cv.line(sat, (src_x, src_y), (dst_x, dst_y), color=255, thickness=1)
    
    save_path = save_pattern.format(idx)
    cv.imwrite(save_path, sat)

In [ ]:
def intersection(p1_satrt, p1_end, p2_start, p2_end):
        
        v1 = np.array(p1_end) - np.array(p1_satrt)
        v2 = np.array(p2_end) - np.array(p2_start)

        v_rel = np.array(p2_start) - np.array(p1_satrt)

        den = v1[1] * v2[0] - v1[0] * v2[1]
        u1 = v1[0] * v_rel[1] - v1[1] * v_rel[0]
        u2 = v2[0] * v_rel[1]  - v2[1] * v_rel[0]

        if den == 0:
            # collinear
            if u1 == 0 and u2 == 0:
                return True
            else:
                return False

        if float(u1) / den < 0 or float(u1) / den > 1 or float(u2) / den < 0 or float(u2) / den > 1:
            return False
        
        return True

In [8]:
import numpy as np

a = np.array(
    [[1, 2],
     [3, 4]])

idx = np.where(a>1)
print(idx)
print(a[a>1])

(array([0, 1, 1]), array([1, 0, 1]))
[2 3 4]
